# [TensorFlow Hub によるテキストの分類: 映画レビュー](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub?hl=ja)

In [ ]:
import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly

import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Data Exploration

In [ ]:
train_data, val_data, test_data = tfds.load(name='imdb_reviews', split=['train[:60%]', 'train[60%:]', 'test'], as_supervised=True)

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

# Modeling

In [ ]:
# Simple use in TensorFlow Hub model

embed = hub.load('https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1')
embeddings = embed(['cat is on the mat', 'dog is in the fog'])
print(embeddings)

In [ ]:
hub_layer = hub.KerasLayer('https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1', input_shape=[], dtype=tf.string, trainable=True)

from tensorflow import keras

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=val_data.batch(512), verbose=1)

In [ ]:
model.evaluate(test_data.batch(512), verbose=2)